In [ ]:
import numpy as np

from amp import ebamp_multimodal
from pca_pack import MultiModalityPCA
from preprocessing import MultiModalityPCADiagnostics
from emp_bayes import ClusterEmpiricalBayes
from hier